# D-Glass Routine Prediction Model

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow.contrib.learn as skflow
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

In [8]:
time_data = pd.read_csv("data/time_data.csv")
time_data.head(15)

,Activity,Day,Time (min)
0,WAKE UP,MONDAY,361.0
1,WAKE UP,TUESDAY,370.0
2,WAKE UP,WEDNESDAY,363.0
3,WAKE UP,THURSDAY,355.0
4,WAKE UP,FRIDAY,346.0
5,WAKE UP,SATURDAY,370.0
6,WAKE UP,SUNDAY,363.0
7,BREAK FAST,MONDAY,485.0
8,BREAK FAST,TUESDAY,495.0
9,BREAK FAST,WEDNESDAY,482.0


In [9]:
# from sklearn.preprocessing import OneHotEncoder

# features = ["Activity", "Day", "Time (min)"]
# one_hot = OneHotEncoder()
# transformer = ColumnTransformer(["one_hot",
#                                 one_hot,
#                                 features])
# transformedX = transformer.fit_transform(time_data)
# transformedX

In [10]:
def get_classification_data():
    data = []
    op = []
    with open("data/classification_data.csv") as dfile:
        for line in dfile.readlines():
            dat = line.split("\n")[0].split(",")
            data.append([int(i) for i in dat[:-1]])
            op.append(int(dat[-1]))
        dfile.close()
    return np.array(data), np.array(op)

def get_regression_data():  
    time_data = {}
    with open("data/time_data.csv") as time_file:
        for line in time_file.readlines():
            td = line.split("\n")[0].split(",")
            if td[0] in time_data:
                if td[1] in time_data[td[0]]:
                    if td[2]:
                        time_data[td[0]][td[1]].append(int(td[2]))
                else:
                    if td[2]:
                        time_data[td[0]][td[1]] = [td[2]]
            else:
                if td[2]:
                    time_data[td[0]] = {td[1]: [td[2]]}
    return time_data

def do_regression(r_op):
    r_ip = [1 for i in r_op]
    
    V = tf.Variable(tf.zeros([1]))
    b = tf.Variable(tf.zeros([1]))

    tf_op = r_ip * V


    # Minimize the mean squared errors.
    loss = tf.reduce_mean(tf.square(tf_op - r_op))
    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = optimizer.minimize(loss)

    init = tf.initialize_all_variables()

    # Launch the graph.
    sess = tf.Session()
    sess.run(init)

    # Fit the line.
    for step in range(301):
        sess.run(train)
    return sess.run(V)[0]

In [11]:
def min_to_time(tm):
    m = tm%60
    h = int(tm/60)
    if h >= 12:
        ap = "PM"
        if h >= 13:
            h -= 12
    else:
        ap = "AM"
    return "\t" + str(h) + ":" + str(m).zfill(2) + " " + ap

def create_prediction_input(act, day):
    acts = ['WAKE UP', 'BREAK FAST', 'M-MEDICATION', 'CHECK TO-DO LIST', 'GO FOR JOB', 'LISTERNMUSIC', 'READING', 'GARDENING', 'LUNCH', 'RETURNFROM JOB', 'EVENING WALK/EXERCISE', 'SHOPPING', 'VISITING RELATIVES', 'DINNER', 'E-MEDICATION', 'SLEEP']

    days = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']
    
    ipact = [1 if i == act else 0 for i in acts]
    ipd = [1 if i == day else 0 for i in days]
    return ipact + ipd

In [12]:
print ("Getting data")
iput, oput = get_classification_data()
act_times = get_regression_data()

print ("Training the machine")
classifier = RandomForestClassifier(n_estimators=len(oput))
classifier.fit(iput, oput)

print ("Learning Complete")

Getting data


NameError: name 'np' is not defined

In [15]:
df1 = pd.DataFrame(act_times)
df1 = df1.drop("Activity", axis=1)
df1 = df1.drop("Day", axis=0)
df1.transpose().head()

,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY,SUNDAY
WAKE UP,"[361, 361, 370, 370, 361, 361, 370, 370, 370, ...","[370, 346, 361, 355, 370, 346, 361, 355, 355, ...","[363, 355, 363, 370, 363, 355, 363, 370, 370, ...","[355, 363, 355, 346, 355, 363, 355, 346, 346, ...","[346, 346, 360, 361, 346, 346, 360, 361, 361, ...","[370, 360, 346, 363, 370, 360, 346, 363, 363, ...","[363, 346, 363, 360, 363, 346, 363, 360, 360, ..."
BREAK FAST,"[485, 485, 495, 495, 485, 485, 495, 495, 495, ...","[495, 465, 485, 475, 495, 465, 485, 475, 475, ...","[482, 475, 482, 495, 482, 475, 482, 495, 495, ...","[475, 482, 475, 465, 475, 482, 475, 465, 465, ...","[465, 465, 480, 485, 465, 465, 480, 485, 485, ...","[495, 480, 465, 482, 495, 480, 465, 482, 482, ...","[482, 465, 482, 480, 482, 465, 482, 480, 480, ..."
M-MEDICATION,"[513, 513, 525, 525, 513, 513, 525, 525, 525, ...","[525, 495, 513, 505, 525, 495, 513, 505, 505, ...","[510, 505, 510, 525, 510, 505, 510, 525, 525, ...","[505, 510, 505, 495, 505, 510, 505, 495, 495, ...","[495, 495, 510, 513, 495, 495, 510, 513, 513, ...","[525, 510, 495, 510, 525, 510, 495, 510, 510, ...","[510, 495, 510, 510, 510, 495, 510, 510, 510, ..."
CHECK TO-DO LIST,"[540, 540, 550, 550, 540, 540, 550, 550, 550, ...","[550, 525, 540, 535, 550, 525, 540, 535, 535, ...","[542, 535, 542, 550, 542, 535, 542, 550, 550, ...","[535, 542, 535, 525, 535, 542, 535, 525, 525, ...","[525, 525, 540, 540, 525, 525, 540, 540, 540, ...","[550, 540, 525, 542, 550, 540, 525, 542, 542, ...","[542, 525, 542, 540, 542, 525, 542, 540, 540, ..."
GO FOR JOB,"[570, 570, 569, 570, 570, 570, 569, 570, 570, ...","[571, 574, 574, 574, 571, 574, 574, 574, 574, ...","[568, 567, 569, 567, 568, 567, 569, 567, 567, ...","[572, 572, 572, 572, 572, 572, 572, 572, 572, ...","[569, 569, 570, 569, 569, 569, 570, 569, 569, ...",NaN,NaN


In [19]:
import warnings
acts = ['WAKE UP', 'BREAK FAST', 'M-MEDICATION', 'CHECK TO-DO LIST', 'GO FOR JOB', 'LISTERNMUSIC', 'READING', 'GARDENING', 'LUNCH', 'RETURNFROM JOB', 'EVENING WALK/EXERCISE', 'SHOPPING', 'VISITING RELATIVES', 'DINNER', 'E-MEDICATION', 'SLEEP']
text = ''
while text != 'stop':
    text = input("\nInput day: ")
    warnings.filterwarnings("ignore")
    if text=='stop':
        break
    wday = text
    graph_data = []
    for ip_act in acts:
        p_ip = create_prediction_input(ip_act, wday)
        prob = classifier.predict([p_ip])[0]
        if prob == 1:
            tm = do_regression([int(i) for i in act_times[ip_act][wday]])
            tm = int(tm)
            gtm = datetime(year = 2016, month = 9, day = 25, hour=int(tm/60), minute=int(tm%60))
            graph_data.append(gtm)
            print (ip_act, min_to_time(tm))
        else:
            gtm = datetime(year = 2016, month = 9, day = 25, hour=0, minute=0)
            graph_data.append(gtm)
            print (ip_act, "\tActivity does not exist")


Input day: MONDAY
WAKE UP 	6:05 AM
BREAK FAST 	8:10 AM
M-MEDICATION 	8:39 AM
CHECK TO-DO LIST 	9:05 AM
GO FOR JOB 	9:29 AM
LISTERNMUSIC 	Activity does not exist
READING 	Activity does not exist
GARDENING 	Activity does not exist
LUNCH 	12:25 PM
RETURNFROM JOB 	4:29 PM
EVENING WALK/EXERCISE 	5:07 PM
SHOPPING 	Activity does not exist
VISITING RELATIVES 	Activity does not exist
DINNER 	8:01 PM
E-MEDICATION 	8:35 PM
SLEEP 	10:05 PM

Input day: stop
